In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
import tensorflow as tf
import time

In [2]:
# Wczytaj:
# - train.csv
# - model.h5

In [3]:
model = tf.keras.models.load_model('model.h5')

In [4]:
def pgd_attack(model, images, labels, epsilon, c, num_iterations, norm_type='Linf', threshold=None):
    """
    Przeprowadza atak PGD z kontrolą normy perturbacji i możliwością wcześniejszego zakończenia.

    Parametry:
    - model: Model Keras lub TensorFlow
    - images: Obrazy wejściowe (batch)
    - labels: Etykiety one-hot dla obrazów
    - epsilon: Maksymalna wielkość perturbacji
    - alpha: Krok każdej iteracji
    - num_iterations: Maksymalna liczba iteracji
    - norm_type: Typ normy ('l0', 'l1', 'l2', 'linf')
    - threshold: Próg zmiany perturbacji dla wcześniejszego zakończenia (opcjonalny)

    Zwraca:
    - Adwersarialne obrazy
    """
    # Inicjalizujemy perturbacje jako kopię obrazów wejściowych
    perturbed_images = tf.identity(images)

    for i in range(num_iterations):
        with tf.GradientTape() as tape:
            tape.watch(perturbed_images)
            predictions = model(perturbed_images)
            loss = tf.keras.losses.categorical_crossentropy(labels, predictions)

        # Obliczamy gradient
        gradient = tape.gradient(loss, perturbed_images)

        # Generowanie perturbacji na podstawie wybranej normy
        if norm_type == 'Linf':
            perturbation = c * tf.sign(gradient)
        elif norm_type == 'L2':
            # Ręczne obliczenie normy L2
            norm = tf.sqrt(tf.reduce_sum(tf.square(gradient), axis=(1, 2, 3), keepdims=True))
            perturbation = c * gradient / (norm + 1e-10)
        else:
            raise ValueError("Nieobsługiwana norma. Użyj 'l0', 'l1', 'l2' lub 'linf'.")

        # Aktualizacja obrazów adwersarialnych
        new_perturbed_images = perturbed_images + perturbation
        new_perturbed_images = tf.clip_by_value(new_perturbed_images, images - epsilon, images + epsilon)  # projekcja
        new_perturbed_images = tf.clip_by_value(new_perturbed_images, 0, 1)  # ograniczenie do zakresu [0,1]

        # Sprawdzenie progu zmiany perturbacji
        if threshold is not None:
            delta = tf.reduce_max(tf.abs(new_perturbed_images - perturbed_images))
            if delta < threshold:
                print(f"Zatrzymanie po {i + 1} iteracjach (zmiana < {threshold})")
                break

        perturbed_images = new_perturbed_images

    return perturbed_images


In [5]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [6]:
data = pd.read_csv('train.csv')

X = data.iloc[:,1:]
Y = data.iloc[:,0]

X_reshaped = X.values.reshape(-1, 28, 28, 1)

In [7]:
x_train , x_test , y_train , y_test = train_test_split(X_reshaped, Y, test_size=0.1)

x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255

datagen.fit(x_train)

In [8]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [9]:
import numpy as np

# Generowanie danych adwersarialnych dla zbioru treningowego
batch_size_train = 63
x_train_adv = []
for i in range(0, len(x_train), batch_size_train):
    batch_images = x_train[i:i+batch_size_train]
    batch_labels = y_train[i:i+batch_size_train]
    print(f"Generuję dane adwersarialne dla treningu: batch {i} - {i+batch_size_train}")
    adv_images = pgd_attack(model, batch_images, batch_labels, epsilon=0.01, c=0.3, num_iterations=50, norm_type="L2")
    x_train_adv.append(adv_images)

x_train_adv = np.concatenate(x_train_adv, axis=0)

# Zapis danych adwersarialnych dla zbioru treningowego
np.savez_compressed("adversarial_train_images.npz", X_train_adv=x_train_adv)
print("Dane adwersarialne dla treningu zostały zapisane jako 'adversarial_train_images.npz'.")

Generuję dane adwersarialne dla treningu: batch 0 - 63
Generuję dane adwersarialne dla treningu: batch 63 - 126
Generuję dane adwersarialne dla treningu: batch 126 - 189
Generuję dane adwersarialne dla treningu: batch 189 - 252
Generuję dane adwersarialne dla treningu: batch 252 - 315
Generuję dane adwersarialne dla treningu: batch 315 - 378
Generuję dane adwersarialne dla treningu: batch 378 - 441
Generuję dane adwersarialne dla treningu: batch 441 - 504
Generuję dane adwersarialne dla treningu: batch 504 - 567
Generuję dane adwersarialne dla treningu: batch 567 - 630
Generuję dane adwersarialne dla treningu: batch 630 - 693
Generuję dane adwersarialne dla treningu: batch 693 - 756
Generuję dane adwersarialne dla treningu: batch 756 - 819
Generuję dane adwersarialne dla treningu: batch 819 - 882
Generuję dane adwersarialne dla treningu: batch 882 - 945
Generuję dane adwersarialne dla treningu: batch 945 - 1008
Generuję dane adwersarialne dla treningu: batch 1008 - 1071
Generuję dane a

KeyboardInterrupt: 

In [ ]:
# Generowanie danych adwersarialnych dla zbioru testowego
batch_size_test = 42
x_test_adv = []
for i in range(0, len(x_test), batch_size_test):
    batch_images = x_test[i:i+batch_size_test]
    batch_labels = y_test[i:i+batch_size_test]
    print(f"Generuję dane adwersarialne dla testów: batch {i} - {i+batch_size_test}")
    adv_images = pgd_attack(model, batch_images, batch_labels, epsilon=0.01, c=0.3, num_iterations=50, norm_type="L2")
    x_test_adv.append(adv_images)

x_test_adv = np.concatenate(x_test_adv, axis=0)

# Zapis danych adwersarialnych dla zbioru testowego
np.savez_compressed("adversarial_test_images.npz", X_test_adv=x_test_adv)
print("Dane adwersarialne dla testów zostały zapisane jako 'adversarial_test_images.npz'.")


## Wczytanie wygenerowanych danych

In [ ]:
# Wczytywanie danych z plików
data_train = np.load("adversarial_train_images.npz")
x_train_adv = data_train["X_train_adv"]
print(f"Wczytano dane adwersarialne dla treningu: {x_train_adv.shape}")

data_test = np.load("adversarial_test_images.npz")
x_test_adv = data_test["X_test_adv"]
print(f"Wczytano dane adwersarialne dla testów: {x_test_adv.shape}")

In [ ]:
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Funkcja do stworzenia modelu pomocniczego
def create_auxiliary_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same',
                     data_format='channels_last', input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Dostosowanie warstwy wyjściowej do klasyfikacji binarnej
    model.add(Dense(1, activation='sigmoid'))  # 1 neuron wyjściowy dla klasyfikacji binarnej

    return model

# Wczytanie lub stworzenie istniejącego modelu bazowego
try:
    base_model = load_model("model.h5")  # Zakładamy, że model istnieje
except OSError:
    base_model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    base_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Zamrożenie warstw istniejącego modelu
for layer in base_model.layers:
    layer.trainable = False

# Tworzenie modelu pomocniczego na podstawie istniejącego
aux_model_extended = Sequential(base_model.layers[:-1])  # Wszystkie warstwy oprócz wyjściowej
aux_model_extended.add(Dense(512, activation='relu'))
aux_model_extended.add(BatchNormalization())
aux_model_extended.add(Dropout(0.25))
aux_model_extended.add(Dense(1024, activation='relu'))
aux_model_extended.add(BatchNormalization())
aux_model_extended.add(Dropout(0.5))
aux_model_extended.add(Dense(1, activation='sigmoid'))  # Wyjście binarne

# Kompilacja modelu pomocniczego
aux_model_extended.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Model pomocniczy na podstawie modelu głównego

In [ ]:
from sklearn.utils import shuffle

# Lista proporcji split
splits = [0.2, 0.4, 0.5, 0.3]

for split in splits:
    num_adv_samples_train = int(len(x_train_adv) * split)

    # Łączenie danych oryginalnych i adwersarialnych dla treningu
    x_train_combined = np.concatenate([x_train, x_train_adv[:num_adv_samples_train]], axis=0)
    y_train_combined = np.concatenate([np.zeros(len(x_train)), np.ones(num_adv_samples_train)], axis=0)  # 0: prawdziwe, 1: adwersarialne
    print(f"Wczytano dane adwersarialne dla treningu: {x_train_combined.shape}")
    print(f"Wczytano dane adwersarialne dla treningu: {y_train_combined.shape}")

    num_adv_samples_test = int(len(x_test_adv) * split)

    # Łączenie danych oryginalnych i adwersarialnych dla testów
    x_test_combined = np.concatenate([x_test, x_test_adv[:num_adv_samples_test]], axis=0)
    y_test_combined = np.concatenate([np.zeros(len(x_test)), np.ones(len(x_test_adv[:num_adv_samples_test]))], axis=0)  # 0: prawdziwe, 1: adwersarialne
    print(f"Wczytano dane adwersarialne dla testu: {x_test_combined.shape}")
    print(f"Wczytano dane adwersarialne dla testu: {y_test_combined.shape}")

    # Mieszanie danych
    x_train_combined, y_train_combined = shuffle(x_train_combined, y_train_combined, random_state=42)


    # Generator danych z augmentacją
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1
    )
    datagen.fit(x_train_combined)

    # Learning Rate Scheduler
    reduce_lr = LearningRateScheduler(lambda epoch: 1e-3 * 0.9 ** epoch)

    # Early Stopping
    early_stopping = EarlyStopping(
        min_delta=0.001,
        patience=20,
        restore_best_weights=True
    )

    # Trening modelu pomocniczego
    print(f"Trening modelu dla split: {split}")
    history = aux_model_extended.fit(
        datagen.flow(x_train_combined, y_train_combined, batch_size=64),
        epochs=50,
        validation_data=(x_test_combined, y_test_combined),
        verbose=1,
        steps_per_epoch=x_train_combined.shape[0] // 64,
        callbacks=[reduce_lr, early_stopping]
    )

    # Zapis modelu
    model_name = f"ADM_model_extended_split_{split}.h5"
    aux_model_extended.save(model_name)
    print(f"Modele zostały zapisane: {model_name}")

